In [70]:
import twitter
import os
import yaml
import re
import time
import tweepy
import pandas as pd
from textblob import TextBlob
from collections import Counter

In [4]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/api_credentials.yml')))

In [5]:
credentials['twitter']

{'consumer_key': 'r9OIAbfLUGTo2Dht0icAQy7Fj',
 'consumer_secret': 'JUZii5KePZcdaz7p3cDYlZ3Eiip6f4c172Ni6yT8cnUtfdxin2',
 'token': '2879880710-dJqMcUZzD9gro3phLgKlDCf1QhNMHApfnaSmY39',
 'token_secret': '4oWj4s4yluaPUmPrXMEYcSoRFavo57TkV7vzdDyt0GMZH'}

# Try Tweepy

In [43]:
#!/usr/bin/env python
# encoding: utf-8

import tweepy #https://github.com/tweepy/tweepy
import csv

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name,company):
	"""Download the last 3240 tweets from a user. Do text processign to remove URLs and the retweets from a user.
	Adapted from https://gist.github.com/yanofsky/5436496"""
	#Twitter only allows access to a users most recent 3240 tweets with this method
	
	#authorize twitter, initialize tweepy
	auth = tweepy.OAuthHandler(credentials['twitter']['consumer_key'], credentials['twitter']['consumer_secret'],)
	auth.set_access_token(credentials['twitter']['token'], credentials['twitter']['token_secret'])
	api = tweepy.API(auth)
	
	#initialize a list to hold all the tweepy Tweets
	alltweets = []	
	
	#make initial request for most recent tweets (200 is the maximum allowed count)
	new_tweets = api.user_timeline(screen_name = screen_name,count=200)
	
	#save most recent tweets
	alltweets.extend(new_tweets)
	
	#save the id of the oldest tweet less one
	oldest = alltweets[-1].id - 1
	
	#keep grabbing tweets until there are no tweets left to grab
	while len(new_tweets) > 0:
		print ("getting tweets before %s" % (oldest))
		
		#all subsiquent requests use the max_id param to prevent duplicates
		new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
		
		#save most recent tweets
		alltweets.extend(new_tweets)
		
		#update the id of the oldest tweet less one
		oldest = alltweets[-1].id - 1
		
		print ("...%s tweets downloaded so far" % (len(alltweets)))
	cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in alltweets] # remove urls
	cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions 
	cleaned_text = [re.sub(r'RT.*','', i, flags=re.MULTILINE) for i in cleaned_text] # delete the retweets
	#transform the tweepy tweets into a 2D array that will populate the csv	
	outtweets = [[tweet.id_str, tweet.created_at, cleaned_text[idx]] for idx,tweet in enumerate(alltweets)]
	
# 	#write the csv	
# 	with open(f'../data/raw/svb_founders/{company}:{screen_name}_tweets.csv', 'w') as f:
# 		writer = csv.writer(f)
# 		writer.writerow(["id","created_at","text"])
# 		writer.writerows(outtweets)
	
	return pd.DataFrame(outtweets,columns=["id","created_at","text"])


In [44]:
df = get_all_tweets('ericries','long_short_term_exchange')

getting tweets before 864489339043979264
...400 tweets downloaded so far
getting tweets before 854835869697556479
...600 tweets downloaded so far
getting tweets before 848287296411418623
...800 tweets downloaded so far
getting tweets before 842410101390565376
...1000 tweets downloaded so far
getting tweets before 837763792276619263
...1200 tweets downloaded so far
getting tweets before 833136755247247360
...1400 tweets downloaded so far
getting tweets before 826044906405638144
...1600 tweets downloaded so far
getting tweets before 795387993741590527
...1798 tweets downloaded so far
getting tweets before 791023336406908928
...1998 tweets downloaded so far
getting tweets before 784760533668900863
...2198 tweets downloaded so far
getting tweets before 776029830815055871
...2398 tweets downloaded so far
getting tweets before 761276197011542015
...2598 tweets downloaded so far
getting tweets before 748148304773427200
...2798 tweets downloaded so far
getting tweets before 742377400177041411


In [82]:
bag_of_words = None
sentences =[]
for row in df.iterrows():
    blob = TextBlob(row[1]['text'])
    blob = blob.lower()
    print(blob)
    break
    tokens = blob.tokenize()
    if bag_of_words == None:
        bag_of_words =  Counter(tokens)
    else:
        words = Counter(tokens)
        for k,v in words.items():
            if k in bag_of_words:
                bag_of_words[k]+=v
            else:
                bag_of_words[k]=v
                
    

very sad that i can't be in 🇬🇧🇬🇧🇬🇧in person this year - but will do my best remotely. see you soon! 


In [81]:
bag_of_words

Counter({'very': 33,
         'sad': 5,
         'that': 147,
         'i': 413,
         'ca': 46,
         "n't": 154,
         'be': 113,
         'in': 163,
         '🇬🇧🇬🇧🇬🇧in': 1,
         'person': 4,
         'this': 348,
         'year': 12,
         '-': 106,
         'but': 99,
         'will': 57,
         'do': 126,
         'my': 120,
         'best': 22,
         'remotely': 1,
         '.': 332,
         'see': 24,
         'you': 285,
         'soon': 20,
         '!': 207,
         'clearly': 6,
         'know': 59,
         'what': 76,
         'they': 45,
         ',': 366,
         'the': 612,
         'hot': 5,
         'takes': 3,
         'are': 82,
         'talking': 4,
         'about': 60,
         'if': 76,
         'it…': 1,
         '``': 95,
         'too': 50,
         'can': 95,
         'like': 66,
         'steph': 1,
         'curry': 3,
         "''": 98,
         'by': 34,
         'trading': 1,
         'cash': 1,
         'for': 237,
         'eq

In [63]:
blob = TextBlob("Very sad that I can't be in 🇬🇧🇬🇧🇬🇧in person this year - but will do my best remotely. See you soon! ")

In [64]:
blob = blob.lower()

In [69]:
blob.words

WordList(['very', 'sad', 'that', 'i', 'ca', "n't", 'be', 'in', '🇬🇧🇬🇧🇬🇧in', 'person', 'this', 'year', 'but', 'will', 'do', 'my', 'best', 'remotely', 'see', 'you', 'soon'])

In [68]:
blob.tokenize()

WordList(['very', 'sad', 'that', 'i', 'ca', "n't", 'be', 'in', '🇬🇧🇬🇧🇬🇧in', 'person', 'this', 'year', '-', 'but', 'will', 'do', 'my', 'best', 'remotely', '.', 'see', 'you', 'soon', '!'])

# Python -Twitter

In [124]:
text = api.GetUserTimeline(screen_name='ashady',count =300, include_rts=False)

In [125]:
cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in text] # remove the urls

In [127]:
cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions

In [128]:
cleaned_text

[" Nice work Eric. Love the analysis. We're building the same using AI. Starting making picks this month  demo day",
 'Flying  and really impressed by the crew. Great service and super friendly!',
 'I just published “Which 500 Startups Demo Day companies does our AI algorithm like?” ',
 'When will AI replace venture capitalists? ',
 'Venture Capital: Are any VC firms using machine learning or AI to help in deal sourcing? ',
 'NYU Stern School of Business  featuring  Stern pride goes both ways :) \n',
 'What are the common traits of successful entrepreneurs? ',
 '6 Traits of Successful Founders ',
 '    thanks Chris!! Means a lot that you like it!',
 'I just published “The Personalities of Successful Founders” ',
 ' ',
 'I just donated to  to help children in need. You can help too!: ',
 ' which one?',
 'Attending a cool session on NLP and machine learning at  ',
 'Complex #DataViz by  on  untangles   business ties ',
 ' yes photos pls!',
 'Just donated ',
 'What it means to stand again

In [129]:
api.GetFollowers(screen_name='ashady')

[User(ID=59840147, ScreenName=kvasnic),
 User(ID=868973790386151424, ScreenName=EsraaEl51255104),
 User(ID=288497184, ScreenName=JTattersfield),
 User(ID=870175789546958848, ScreenName=ADo7a93),
 User(ID=24961055, ScreenName=mattbiehl),
 User(ID=744150240136044544, ScreenName=djkingassassin),
 User(ID=711273343396278272, ScreenName=JaydaBF),
 User(ID=351100725, ScreenName=Dylan_Bostic),
 User(ID=864015366946775040, ScreenName=RehamMohamedAh7),
 User(ID=296320946, ScreenName=SAPTradeGeek),
 User(ID=866383033292771328, ScreenName=TurkiBinA2),
 User(ID=865124920598179840, ScreenName=PeaceHackMENA),
 User(ID=799012517129269248, ScreenName=MuhammedEzzat10),
 User(ID=1879978364, ScreenName=ryanhintze),
 User(ID=805559119566565376, ScreenName=ask1angel),
 User(ID=3784808057, ScreenName=bouigoudal),
 User(ID=25217355, ScreenName=_moustafa),
 User(ID=266420236, ScreenName=treyptrsn),
 User(ID=2707473229, ScreenName=DreamsdotBuild),
 User(ID=556259902, ScreenName=OmarAbdElRhmaan),
 User(ID=86024

In [108]:
def twitter_text(username):
    """Return the last 200 tweets' text from the given username"""
    text = api.GetUserTimeline(screen_name=username,count =300, include_rts=False)
    cleaned_text = [re.sub(r'http[s]?:\/\/.*[\W]*', '', i.text, flags=re.MULTILINE) for i in text] # remove the urls
    cleaned_text = [re.sub(r'@[\w]*', '', i, flags=re.MULTILINE) for i in cleaned_text] # remove the @twitter mentions
    return cleaned_text

In [130]:
twitter_text('olivercameron')

['It took ~4 years but they did it! ',
 'Surely radar and cameras is preferable to LIDAR? Primarily for range &amp; cost: ',
 '   Definitely!',
 '  did all the good work there 👌',
 'We recently rebuilt our entire  controls stack for silky smooth rides. Lots of related good stuff in here! ',
 ' Great decision!  and the team are building a world-changing product.',
 'Awesome opportunity. ',
 '😱 ',
 '   You can in the MKZ, not in the Fusion.',
 'A template for building a great company, courtesy of . ',
 ' Pumped to see what you build next! Just my personal opinion, but I would try to cure cancer with dinosaurs.',
 '😶  ',
 '    ',
 '   is the exact opposite, prefers Antarctic winds in the car.',
 'To be clear: this is a bug in the  app, not the actual car temperature ☀️ ',
 '🤔 ',
 'Insane growth! ',
 'Fun visit to  earlier. Mapping (and all that comes with it) is a huge pain-point in autonomous vehicles… ',
 'Awesome interview. ',
 '      Can you imagine?',
 'I should mention, we plan to o

In [ ]:
def twitter_followers(username):
    """REturn the number of followers for the given user. Need to sleep to not exceed the rate limit."""
    for num in range(1_000_000):
        if num %500 ==0:
            time.sleep(5)
            